In [1]:
pip install spotipy

     |████████████████████████████████| 153 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 4.8 MB/s eta 0:00:011
ERROR: botocore 1.18.4 has requirement urllib3<1.26,>=1.20; python_version != "3.4", but you'll have urllib3 1.26.4 which is incompatible.
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.9
    Uninstalling urllib3-1.25.9:
      Successfully uninstalled urllib3-1.25.9
  Attempting uninstall: requests
    Found existing installation: requests 2.24.0
    Uninstalling requests-2.24.0:
      Successfully uninstalled requests-2.24.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install confluent-kafka

     |████████████████████████████████| 2.7 MB 1.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


Imports

In [65]:
from confluent_kafka import Consumer, Producer
import os
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy
from math import sqrt
from sklearn.metrics.pairwise import cosine_similarity as cosine_similarity
from pyspark.sql.types import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import PCA
from pyspark.ml.evaluation import ClusteringEvaluator
import pyspark
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[4]").getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [33]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
SPOTIPY_CLIENT_ID='ff464bf1719f4878a4c6b078329878fb'
SPOTIPY_CLIENT_SECRET='ccd5c137ece94d78b801a283b50f0340'
auth_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [63]:
def getrows(df, rownums=None):
    return df.rdd.zipWithIndex().filter(lambda x: x[1] > rownums[0] and x[1] < rownums[1]).map(lambda x: x[0])

def similarity_measure(record):
    prediction = record['prediction']
    #songs features
    try:
        features_vec = numpy.asarray(list(record['features'])).reshape(1, -1)
        cluster_info = spark \
                    .sql(f'SELECT feature_one, feature_two, track_name, artist_name FROM Predictions WHERE prediction={prediction}') \
                    .toPandas()
        cluster_vec = numpy.asarray(cluster_info[['feature_one', 'feature_two']])
        output = cosine_similarity(features_vec, cluster_vec).T
        cluster_info['similarity'] = pd.Series(output.tolist()).values
    
        sim_DF = spark.createDataFrame(cluster_info)
        recommed = sim_DF.sort(sim_DF.similarity.desc()).select(['track_name', 'artist_name']).take(10)
        return recommend
    
    finally:
        
        return 0
                
def read_and_normalize(streamData):
    #Use API to get track features
    streamData_features = sc.parallelize(streamData) \
                          .map(lambda track: (sp.audio_features(track[0].strip())[0], track[0], track[1])).collect()
    
    #For each track extract the relevant data
    cols = ['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']
    for i, (features, uri, name) in enumerate(streamData_features):
        features = {column: value for (column, value) in features.items() if column in cols}
        features['track_uri'] = uri
        features['track_name'] = name
        streamData_features[i] = features
   
    #return the to to RDD
    streamData_RDD = sc.parallelize(streamData_features).persist()
    
    #Extract the values from each feature and normalize them
    extract_RDD = streamData_RDD.map(lambda record: tuple(val for val in list(record.values()))).persist()
    normal_streamData_RDD = extract_RDD.map(lambda x: [x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],float(x[8])/100,x[9],x[10]]) \
                        .map(lambda x: ([float(i/np.linalg.norm(x[:-2])) for i in x[:-2]], tuple(x[-2:]))) \
                        .map(lambda x: (*x[0], x[1][0], x[1][1] ))
    
    #Build a Dataframe from the normalized RDD
    schema = StructType([
    StructField('danceability', FloatType(), False),
    StructField('energy', FloatType(), False),
    StructField('loudness', FloatType(), False),
    StructField('speechiness', FloatType(), False),
    StructField('acousticness', FloatType(), False),
    StructField('instrumentalness', FloatType(), False),
    StructField('liveness', FloatType(), False),
    StructField('valence', FloatType(), False),
    StructField('tempo', FloatType(), False),
    StructField('track_uri', StringType(), False),
    StructField('track_name', StringType(), False)
    ])
    normal_streamData_DF = normal_streamData_RDD.map(lambda x: x).toDF(schema=schema)
    normal_streamData_feature = normal_streamData_DF.rdd \
                                .map(lambda row: Row(Vectors.dense([row[0], row[1], row[2], row[3], row[4],row[5],row[6],row[7],row[8]]), row[9], row[10])).toDF(["features", "track_uri", "track_name"]).persist()
    return normal_streamData_feature

# Prepare train data to Kmeans

In [6]:
schema = StructType([
    StructField('index', StringType(), False),
    StructField('danceability', DoubleType(), False),
    StructField('energy', DoubleType(), False),
    StructField('key', IntegerType(), False),
    StructField('loudness', DoubleType(), False),
    StructField('mode', IntegerType(), False),
    StructField('speechiness', DoubleType(), False),
    StructField('acousticness', DoubleType(), False),
    StructField('instrumentalness', DoubleType(), False),
    StructField('liveness', DoubleType(), False),
    StructField('valence', DoubleType(), False),
    StructField('tempo', DoubleType(), False),
    StructField('type', StringType(), False),
    StructField('id', StringType(), False),
    StructField('uri', StringType(), False),
    StructField('track_href', StringType(), False),
    StructField('analysis_url', StringType(), False),
    StructField('duration_ms', IntegerType(), False),
    StructField('time_signature', IntegerType(), False),
    StructField('popularity', IntegerType(), False),
    StructField('track_name', StringType(), False),
    StructField('artist_name', StringType(), False),
])

tracks_df = spark.read.csv("sp_dataset.csv",schema=schema)
existing = tracks_df.columns
new_names = ['index','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','type','id','uri','track_href','analysis_url','duration_ms','time_signature','popularity','track_name','artist_name']
for old , new in zip(existing,new_names):
    tracks_df = tracks_df.withColumnRenamed(old,new)

In [7]:
train_df = getrows(tracks_df,[1,11000]).toDF()
audio_trainRDD = train_df.rdd.map(lambda x: (x['danceability'],x['energy'],x['key'],x['loudness'],x['mode'],x['speechiness'],x['acousticness'],x['instrumentalness'],x['liveness'],x['valence'],x['tempo']))
# divide tempo by 100 then normalize the data
audio_normal_trainRDD = audio_trainRDD.map(lambda x: (x[0],x[1],x[3],x[5],x[6],x[7],x[8],x[9],float(x[10])/100))\
                       .map(lambda x: [float(i/np.linalg.norm(x)) for i in x])

In [8]:
track_namesRDD = train_df.rdd.map(lambda track: (track['uri'], track['track_name'], track['artist_name']))
track_names = track_namesRDD.collect()

In [9]:
schema = StructType([
    StructField('danceability', FloatType(), False),
    StructField('energy', FloatType(), False),
    StructField('loudness', FloatType(), False),
    StructField('speechiness', FloatType(), False),
    StructField('acousticness', FloatType(), False),
    StructField('instrumentalness', FloatType(), False),
    StructField('liveness', FloatType(), False),
    StructField('valence', FloatType(), False),
    StructField('tempo', FloatType(), False),
])

audio_normal_train_DF = audio_normal_trainRDD.map(lambda x: x).toDF(schema=schema)
normal_train_df = audio_normal_train_DF.rdd.map(lambda row: Row(Vectors.dense([row[0], row[1], row[2], row[3], row[4],row[5],row[6],row[7],row[8]]))).toDF(["features"])


Implement PCA on the train data to extract 2 features

In [10]:
#trainData ,testData = my_pca(normal_train_df, normal_test_df, 2)
pca = PCA(k=2, inputCol = 'features' , outputCol = 'pcaFeatures')
pca_model = pca.fit(normal_train_df)
trainData = pca_model.transform(normal_train_df).select("pcaFeatures")
trainData = trainData.rdd.map(lambda row: row).toDF(["features"])

Train on Kmeans Algorithmn 

In [12]:
kmeans = KMeans().setK(6).setSeed(1)
kmeans_model = kmeans.fit(trainData)
predictionsTrain = kmeans_model.transform(trainData)

In [13]:
zipped_RDD = predictionsTrain.rdd.zip(track_namesRDD)

In [14]:
zipped_RDD.take(1)

[(Row(features=DenseVector([0.1822, -0.0098]), prediction=1),
  ('spotify:track:4mKjZaJd7w1UEeGOqHCcof', 'True Love', 'WE ARE TWIN'))]

In [15]:
zipped_DF = zipped_RDD.map(lambda row: (row[1][0],row[1][1],row[1][2],float(row[0][0][0]),float(row[0][0][1]),row[0][1])).toDF(['uri','track_name','artist_name','feature_one','feature_two','prediction'])
#Create SQl table for later use
zipped_DF.createOrReplaceTempView("Predictions")

# Kafka from here

In [66]:
p = Producer({'bootstrap.servers': 'localhost:9092'})

conf = {'bootstrap.servers': "localhost:9092",
        'group.id': "spotify",
        'auto.offset.reset': 'smallest'}

consumer = Consumer(conf)
batch_size = 0
batch_msg = []
batch_id = 1

In [67]:
try:
    consumer.subscribe(['spotify-recommend_v1'])

    while True:
        msg = consumer.poll(timeout=1.0)
        
        if msg is None: 
            continue

        
        else:
            
            if msg.key().decode("utf-8") == '0':
                    print(msg.value().decode("utf-8").split(" "))
                    break
                    
            if batch_size < 10:
                batch_size += 1
                
                msg = msg.value().decode("utf-8").split(",")
                batch_msg.append(tuple(msg))
            else:
                normal_streamData_feature = read_and_normalize(batch_msg)
                #compress the data into 2 features only and run on Kmeans
                pca_data = pca_model.transform(normal_streamData_feature.select("features")).select("pcaFeatures")
                to_kmeans_data = pca_data.rdd.map(lambda row: row).toDF(["features"])

                stream_predictions = kmeans_model.transform(to_kmeans_data)

                #Zip 
                track_info = normal_streamData_feature.select(["track_uri", "track_name"])
                schema = StructType(stream_predictions.schema.fields + track_info.schema.fields)
                combined_data = stream_predictions.rdd.zip(track_info.rdd).map(lambda x: x[0]+x[1]).toDF(schema)
                combined_data = combined_data.withColumn('recommendations', lit(0)).collect()
                
                #Add similarity to stream predoctions
                all_recommendations = []
                for row in combined_data:
                    recommendations = similarity_measure(row)
                    if recommendations != 0:
                        new_row = Row(track_name=row['track_name'], recommend=recommendations, track_uri=row['track_uri'])
                        all_recommendations.append(new_row)
                    else:
                        continue
                        
                
                print(f'For batch id {batch_id} ----------------------')
                for recm in all_recommendations:
                    t_name = recm['track_name']
                    recm_lst = recm['recommend']
                    msg = " ".join(f'{recm_lst}')
                    p.produce("recommend-result", key = f'{batch_id},{t_name}', value = msg)
                    
                    
                    
                batch_id += 1
                batch_size = 0
                batch_msg = []
finally:
    p.produce("recommend-result", key = 'stop', value = '0')
    #Close down consumer to commit final offsets.
    consumer.close()

For batch id 1 ----------------------
For batch id 2 ----------------------
For batch id 3 ----------------------
For batch id 4 ----------------------
For batch id 5 ----------------------
For batch id 6 ----------------------
For batch id 7 ----------------------
For batch id 8 ----------------------
For batch id 9 ----------------------
For batch id 10 ----------------------
For batch id 11 ----------------------
For batch id 12 ----------------------
For batch id 13 ----------------------
For batch id 14 ----------------------
For batch id 15 ----------------------
For batch id 16 ----------------------
For batch id 17 ----------------------
For batch id 18 ----------------------
For batch id 19 ----------------------
For batch id 20 ----------------------
For batch id 21 ----------------------
For batch id 22 ----------------------


KeyboardInterrupt: 

In [18]:
test_schema = StructType([StructField('track_uri', StringType(), False),StructField('track_name', StringType(), False)])
test_data = spark.read.format("csv").schema(test_schema).load("DataToStream.csv")

In [19]:
streamData = test_data.rdd.map(lambda s: (s['track_uri'], s['track_name'])).collect()[:10]
streamData

[('spotify:track:0xBOAziRHVjPXOsigGpPHh',
  'How to Save a Life (Running Mix) [140 BPM]'),
 ('spotify:track:25eA6wrjKEVtJb2LGt9ii3',
  "I Don't Wanna Miss a Thing (Runner Remix)"),
 ('spotify:track:3XDHBRyJQWx1zCGhdsQRP4', 'Basket Case - Workout Remix'),
 ('spotify:track:6sumVx6Qrsnerf4KRvA4Va',
  'Only Girl (In the World) [Running Mix] [140 BPM]'),
 ('spotify:track:1q6JBBjlE4dTGKzntIZS2n',
  'Body Like a Back Road - Workout Mix'),
 ('spotify:track:5qpLBTpYlaBdSMRRhfvWfw', 'This Is How We Roll - Workout Mix'),
 ('spotify:track:5aU4XjyMB9eFw67lGSrSQ3', 'H.O.L.Y. - Workout Mix'),
 ('spotify:track:1KJq03F5FI4SuplXCIJaWD', 'Better Man - Workout Mix'),
 ('spotify:track:4XQ0WbhvqOZG33PqO6UEty', 'Homegrown - Workout Mix'),
 ('spotify:track:6R5wnsCBZZRDxE0P1FCGsk',
  'Setting the World on Fire - Workout Mix')]

Use API to get track features

In [20]:
streamData_features = sc.parallelize(streamData).map(lambda track: (sp.audio_features(track[0])[0], track[0], track[1])).collect()

In [21]:
streamData_features[0]

({'danceability': 0.675,
  'energy': 0.99,
  'key': 6,
  'loudness': -7.654,
  'mode': 1,
  'speechiness': 0.0463,
  'acousticness': 0.0862,
  'instrumentalness': 0.0159,
  'liveness': 0.341,
  'valence': 0.645,
  'tempo': 140.002,
  'type': 'audio_features',
  'id': '0xBOAziRHVjPXOsigGpPHh',
  'uri': 'spotify:track:0xBOAziRHVjPXOsigGpPHh',
  'track_href': 'https://api.spotify.com/v1/tracks/0xBOAziRHVjPXOsigGpPHh',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0xBOAziRHVjPXOsigGpPHh',
  'duration_ms': 305787,
  'time_signature': 4},
 'spotify:track:0xBOAziRHVjPXOsigGpPHh',
 'How to Save a Life (Running Mix) [140 BPM]')

For each track extract the relevant data

In [22]:
cols = audio_normal_train_DF.columns
for i, (features, uri, name) in enumerate(streamData_features):
    features = {column: value for (column, value) in features.items() if column in cols}
    features['track_uri'] = uri
    features['track_name'] = name
    streamData_features[i] = features
   


#return the to to RDD
streamData_RDD = sc.parallelize(streamData_features).persist()

In [23]:
streamData_RDD.take(1)

[{'danceability': 0.675,
  'energy': 0.99,
  'loudness': -7.654,
  'speechiness': 0.0463,
  'acousticness': 0.0862,
  'instrumentalness': 0.0159,
  'liveness': 0.341,
  'valence': 0.645,
  'tempo': 140.002,
  'track_uri': 'spotify:track:0xBOAziRHVjPXOsigGpPHh',
  'track_name': 'How to Save a Life (Running Mix) [140 BPM]'}]

Extract the values from each feature and normalize them

In [24]:
extract_RDD = streamData_RDD.map(lambda record: tuple(val for val in list(record.values()))).persist()
normal_streamData_RDD = extract_RDD.map(lambda x: [x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],float(x[8])/100,x[9],x[10]]) \
                        .map(lambda x: ([float(i/np.linalg.norm(x[:-2])) for i in x[:-2]], tuple(x[-2:]))) \
                        .map(lambda x: (*x[0], x[1][0], x[1][1] ))

In [25]:
normal_streamData_RDD.take(1)

[(0.08536672628476583,
  0.1252045318843232,
  -0.9679954414571815,
  0.005855525077014307,
  0.010901647119624908,
  0.002010860663596706,
  0.043126005426822434,
  0.08157264956099845,
  0.1770594431602931,
  'spotify:track:0xBOAziRHVjPXOsigGpPHh',
  'How to Save a Life (Running Mix) [140 BPM]')]

Build a Dataframe from the normalized RDD

In [26]:
schema = StructType([
    StructField('danceability', FloatType(), False),
    StructField('energy', FloatType(), False),
    StructField('loudness', FloatType(), False),
    StructField('speechiness', FloatType(), False),
    StructField('acousticness', FloatType(), False),
    StructField('instrumentalness', FloatType(), False),
    StructField('liveness', FloatType(), False),
    StructField('valence', FloatType(), False),
    StructField('tempo', FloatType(), False),
    StructField('track_uri', StringType(), False),
    StructField('track_name', StringType(), False)
])


normal_streamData_DF = normal_streamData_RDD.map(lambda x: x).toDF(schema=schema)
normal_streamData_feature = normal_streamData_DF.rdd.map(lambda row: Row(Vectors.dense([row[0], row[1], row[2], row[3], row[4],row[5],row[6],row[7],row[8]]), row[9], row[10])).toDF(["features", "track_uri", "track_name"]).persist()

Use PCA on the streamed data

In [27]:
#compress the data into 2 features only and run on Kmeans
pca_data = pca_model.transform(normal_streamData_feature.select("features")).select("pcaFeatures")
to_kmeans_data = pca_data.rdd.map(lambda row: row).toDF(["features"])

Insert the to Kmeans

In [28]:
stream_predictions = kmeans_model.transform(to_kmeans_data)

In [29]:
stream_predictions

features,prediction
[0.14088983892811...,2
[0.09532336953450...,1
[0.12267709572253...,2
[0.21706694050642...,0
[0.12424063091373...,2
[0.11195274526271...,2
[0.09217066861381...,5
[0.02047888859523...,5
[0.11943586201788...,2
[0.10809742419870...,1


In [48]:
track_info = normal_streamData_feature.select(["track_uri", "track_name"])
schema = StructType(stream_predictions.schema.fields + track_info.schema.fields)
combined_data = stream_predictions.rdd.zip(track_info.rdd).map(lambda x: x[0]+x[1]).toDF(schema)

In [49]:
combined_data = combined_data.withColumn('recommendations', lit(0)).collect()

In [61]:
combined_data[0]

Row(features=DenseVector([0.1409, 0.0507]), prediction=2, track_uri='spotify:track:0xBOAziRHVjPXOsigGpPHh', track_name='How to Save a Life (Running Mix) [140 BPM]', recommendations=0)

In [71]:
all_recommendations = []
for row in combined_data:
    recommendations = similarity_measure(row)
    new_row = Row(track_name=row['track_name'], recommend=recommendations, track_uri=row['track_uri'])
    all_recommendations.append(new_row)